In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv


In [2]:
#inports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
# Import the necessary libraries
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier


In [3]:
df_train = pd.read_csv('/kaggle/input/playground-series-s5e1/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')
df_sub = pd.read_csv('/kaggle/input/playground-series-s5e1/sample_submission.csv')

In [4]:
#Extract month, day, and year from date column
df_train['date'] = pd.to_datetime(df_train['date'])

# Extract day, month, and year
df_train['day'] = df_train['date'].dt.day
df_train['month'] = df_train['date'].dt.month
df_train['year'] = df_train['date'].dt.year


#Doing same processes on the test dataset
df_test['date'] = pd.to_datetime(df_test['date'])
df_test['day'] = df_test['date'].dt.day
df_test['month'] = df_test['date'].dt.month
df_test['year'] = df_test['date'].dt.year

In [5]:
df_train.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold', 'day', 'month',
       'year'],
      dtype='object')

In [6]:
# Deal with NAN's
# Replace NaN values with column means
df_train['num_sold'].fillna(df_train['num_sold'].mean(), inplace=True)
3

<ipython-input-6-f4ef0fc19425>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['num_sold'].fillna(df_train['num_sold'].mean(), inplace=True)


3

In [7]:
# encoding categorical fields
df_train_encoded = pd.get_dummies(df_train)

In [8]:
df_train_encoded.columns

Index(['id', 'date', 'num_sold', 'day', 'month', 'year', 'country_Canada',
       'country_Finland', 'country_Italy', 'country_Kenya', 'country_Norway',
       'country_Singapore', 'store_Discount Stickers',
       'store_Premium Sticker Mart', 'store_Stickers for Less',
       'product_Holographic Goose', 'product_Kaggle', 'product_Kaggle Tiers',
       'product_Kerneler', 'product_Kerneler Dark Mode'],
      dtype='object')

In [9]:
#print(df_train.head())
#features = ['country', 'store', 'product'
#            , 'day', 'month','year']
X = df_train_encoded.drop(['id', 'date', 'num_sold'], axis=1)
y = df_train_encoded['num_sold']

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
SEED = 23


model = RandomForestClassifier()
model.fit(X, y)

# Create a DataFrame to store feature importances
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
feature_importances.sort_values(by='Importance', ascending=False, inplace=True)

# Visualize the top N important features
top_n = 10
plt.barh(feature_importances['Feature'][:top_n], feature_importances['Importance'][:top_n])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [10]:
# Create Gradient Bosoting Regressor
gbr = GradientBoostingRegressor(loss='absolute_error',
                                learning_rate=0.1,
                                n_estimators=300,
                                max_depth = 1, 
                                random_state = SEED,
                                max_features = 5)

In [11]:
gbr.fit(train_X, train_y)

GradientBoostingRegressor(loss='absolute_error', max_depth=1, max_features=5,
                          n_estimators=300, random_state=23)

In [12]:
pred_y = gbr.predict(val_X)
test_rmse = mean_squared_error(val_y, pred_y) ** (1 / 2)
print('Root mean Square error: {:.2f}'.format(test_rmse))


Root mean Square error: 394.50


In [13]:
sklearn.metrics.mean_absolute_percentage_error(val_y, pred_y)

3.196219081568321

test_x = df_test[features]
pred = sticker_model.predict(test_x)

#import pandas as pd
#test = pd.read_csv("test.csv")
df_test["num_sold"] = pred
df_test[["id","num_sold"]].to_csv("submission.csv",index=False)